In [1]:
import pandas as pd

In [2]:
dados_treino = pd.read_csv("/content/drive/MyDrive/curso_word2vec/treino.csv")
dados_treino.sample(5)

,title,text,date,category,subcategory,link
74012,Governo decidirá se candidato que se diz negro...,"O Ministério do Planejamento publicou, nesta t...",2016-02-08,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2016/08...
30540,"'Aquele sangue era meu', diz mãe que viu filho...","RESUMO Ao voltar do mercado, Sheila Cristina d...",2016-06-14,cotidiano,NaN,http://www1.folha.uol.com.br/cotidiano/2016/06...
80248,As mulheres que usam burquíni não têm opção,"No domingo de manhã, na praia, vi uma senhora ...",2016-08-30,colunas,alexandrevidalporto,http://www1.folha.uol.com.br/colunas/alexandre...
3424,Al Qaeda no Iêmen anuncia morte de seu princip...,A Al Qaeda na Península Arábica (AQPA) anuncio...,2015-04-14,mundo,NaN,http://www1.folha.uol.com.br/mundo/2015/04/161...
21495,São Paulo compra mais 40% dos direitos do vola...,O São Paulo confirmou nesta terça-feira (8) a ...,2015-08-12,esporte,NaN,http://www1.folha.uol.com.br/esporte/2015/12/1...


In [3]:
!python -m spacy download pt_core_news_sm

2023-05-13 23:52:47.525481: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-13 23:52:49.701283: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 27.9 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [4]:
import spacy
nlp = spacy.load("pt_core_news_sm")

In [5]:
texto = "Rio de Janeiro é um cidade maravilhosa"
doc = nlp(texto)

In [6]:
type(doc)

spacy.tokens.doc.Doc

In [7]:
doc[1].is_stop

True

In [8]:
textos_para_tratamento = (titulos.lower() for titulos in dados_treino["title"])


In [9]:
def trata_textos(doc):
  tokens_validos = []
  for token in doc:
    e_valido = not token.is_stop and token.is_alpha
    if e_valido:
      tokens_validos.append(token.text)

  if len(tokens_validos) > 2:
    return " ".join(tokens_validos)

In [10]:
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [11]:
type(nlp)

spacy.lang.pt.Portuguese

In [12]:
texto = "Rio de Janeiro 123123321 **** @#$ é uma cidade maravilhosa!"
doc = nlp(texto)
trata_textos(doc)

'Rio Janeiro cidade maravilhosa'

In [13]:
from time import time

t0 = time()
textos_tratados = [trata_textos(doc) for doc in nlp.pipe(textos_para_tratamento,
                                                         batch_size = 1000,
                                                         n_process = -1)]

tf = time() - t0

print(tf/60)

4.28595195611318


In [14]:
titulos_tratados = pd.DataFrame({"titulo": textos_tratados})
titulos_tratados.head()

,titulo
0,polêmica marine le pen abomina negacionistas h...
1,macron le pen turno frança revés siglas tradic...
2,apesar larga vitória legislativas macron terá ...
3,governo antecipa balanço alckmin anuncia queda...
4,queda maio atividade econômica sobe junho bc


In [16]:
from gensim.models import Word2Vec


w2v_modelo = Word2Vec(sg=0,
                      window=2,
                      vector_size=300,
                      min_count=5,
                      alpha=0.03,
                      min_alpha=0.0007,
                      workers=-1)


In [17]:
w2v_modelo

In [18]:
print(len(titulos_tratados))
titulos_tratados = titulos_tratados.dropna().drop_duplicates()
print(len(titulos_tratados))

90000
84466


In [19]:
lista_lista_tokens = [titulo.split(" ") for titulo in titulos_tratados.titulo]

In [20]:
import logging

logging.basicConfig(format="%(asctime)s : - %(message)s", level = logging.INFO)

w2v_modelo = Word2Vec(sg=0,
                      window=2,
                      vector_size=300,
                      min_count=5,
                      alpha=0.03,
                      min_alpha=0.0007,
                      workers=-1)

w2v_modelo.build_vocab(lista_lista_tokens, progress_per=5000)

In [21]:
dir(w2v_modelo)


['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_corpus_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_next_alpha',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_raw_word_count',
 '_save_specials',
 '_scan_vocab',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'add_lifecycle_event',
 'add_null_word',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'cbow_mean',
 'comment',
 'compute_loss',
 'corpus_count',
 

In [22]:
w2v_modelo.corpus_count

84466

In [54]:
w2v_modelo.train(lista_lista_tokens, 
                 total_examples=w2v_modelo.corpus_count,
                 epochs = 30)

(0, 0)

In [53]:
w2v_modelo.wv.most_similar("google")

[('socióloga', 0.2489001303911209),
 ('civis', 0.24015411734580994),
 ('cubs', 0.22971174120903015),
 ('saga', 0.22959476709365845),
 ('bolha', 0.2146669179201126),
 ('legaliza', 0.2145625203847885),
 ('ficou', 0.1990867555141449),
 ('previdenciário', 0.19682806730270386),
 ('facas', 0.1933879405260086),
 ('fome', 0.19141922891139984)]

In [25]:
w2v_modelo.wv.most_similar("microsoft")

[('gastam', 0.22926442325115204),
 ('tecidos', 0.21279968321323395),
 ('hostis', 0.19858402013778687),
 ('espalhar', 0.19822123646736145),
 ('agita', 0.19374921917915344),
 ('posa', 0.19213968515396118),
 ('abrigou', 0.19146621227264404),
 ('gravar', 0.18892936408519745),
 ('lage', 0.18454720079898834),
 ('mecanismo', 0.1834438294172287)]

In [26]:
w2v_modelo.wv.most_similar("barcelona")

[('pacers', 0.23627379536628723),
 ('usado', 0.2293020784854889),
 ('embaixador', 0.2199876308441162),
 ('seca', 0.19825494289398193),
 ('teaser', 0.19692440330982208),
 ('absolvição', 0.19333983957767487),
 ('parques', 0.18624509871006012),
 ('rejeitou', 0.18389984965324402),
 ('tomba', 0.1769121289253235),
 ('baltimore', 0.17651571333408356)]

In [27]:
w2v_modelo.wv.most_similar("messi")

[('vigorosa', 0.22851061820983887),
 ('pais', 0.20737206935882568),
 ('notável', 0.20155011117458344),
 ('forem', 0.1949884444475174),
 ('rogério', 0.1926041543483734),
 ('partes', 0.18923793733119965),
 ('incorporar', 0.17882360517978668),
 ('adiou', 0.17814774811267853),
 ('oktoberfest', 0.1779433786869049),
 ('ana', 0.17717429995536804)]

In [28]:
w2v_modelo.wv.most_similar("gm")

[('suassuna', 0.2535056471824646),
 ('reforçam', 0.2147754728794098),
 ('simpatia', 0.19965259730815887),
 ('elogia', 0.19551709294319153),
 ('gêmeos', 0.19317837059497833),
 ('remédio', 0.19201937317848206),
 ('nintendo', 0.1902330368757248),
 ('importância', 0.18855606019496918),
 ('about', 0.187388077378273),
 ('corrida', 0.18647247552871704)]

In [51]:
#Treinamento do modelo Skip-Gram
w2v_modelo_sg = Word2Vec(sg=1,
                         window=5,
                         vector_size=300,
                         min_count=5,
                         alpha=0.03,
                         min_alpha=0.0007,
                         workers=-1)

w2v_modelo_sg.build_vocab(lista_lista_tokens, progress_per=5000)

w2v_modelo_sg.train(lista_lista_tokens, 
                 total_examples=w2v_modelo_sg.corpus_count,
                 epochs = 30)

(0, 0)

In [52]:
w2v_modelo_sg.wv.most_similar("google")

[('socióloga', 0.2489001303911209),
 ('civis', 0.24015411734580994),
 ('cubs', 0.22971174120903015),
 ('saga', 0.22959476709365845),
 ('bolha', 0.2146669179201126),
 ('legaliza', 0.2145625203847885),
 ('ficou', 0.1990867555141449),
 ('previdenciário', 0.19682806730270386),
 ('facas', 0.1933879405260086),
 ('fome', 0.19141922891139984)]

In [31]:
w2v_modelo.wv.most_similar("google")

[('socióloga', 0.2489001303911209),
 ('civis', 0.24015411734580994),
 ('cubs', 0.22971174120903015),
 ('saga', 0.22959476709365845),
 ('bolha', 0.2146669179201126),
 ('legaliza', 0.2145625203847885),
 ('ficou', 0.1990867555141449),
 ('previdenciário', 0.19682806730270386),
 ('facas', 0.1933879405260086),
 ('fome', 0.19141922891139984)]

In [55]:

w2v_modelo_sg.wv.most_similar("gm")

[('suassuna', 0.2535056471824646),
 ('reforçam', 0.2147754728794098),
 ('simpatia', 0.19965259730815887),
 ('elogia', 0.19551709294319153),
 ('gêmeos', 0.19317837059497833),
 ('remédio', 0.19201937317848206),
 ('nintendo', 0.1902330368757248),
 ('importância', 0.18855606019496918),
 ('about', 0.187388077378273),
 ('corrida', 0.18647247552871704)]

In [56]:
w2v_modelo.wv.most_similar("gm")

[('suassuna', 0.2535056471824646),
 ('reforçam', 0.2147754728794098),
 ('simpatia', 0.19965259730815887),
 ('elogia', 0.19551709294319153),
 ('gêmeos', 0.19317837059497833),
 ('remédio', 0.19201937317848206),
 ('nintendo', 0.1902330368757248),
 ('importância', 0.18855606019496918),
 ('about', 0.187388077378273),
 ('corrida', 0.18647247552871704)]

In [34]:
w2v_modelo.wv.save_word2vec_format("/content/drive/My Drive/curso_word2vec/modelo_cbow.txt", binary=False)
w2v_modelo_sg.wv.save_word2vec_format("/content/drive/My Drive/curso_word2vec/modelo_skipgram.txt", binary=False)
     

In [35]:
!python -m spacy download pt_core_news_sm

2023-05-13 23:59:13.914111: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 69.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


In [36]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors

w2v_modelo_cbow = KeyedVectors.load_word2vec_format("/content/drive/My Drive/curso_word2vec/modelo_cbow.txt")
w2v_modelo_sg = KeyedVectors.load_word2vec_format("/content/drive/My Drive/curso_word2vec/modelo_skipgram.txt")
artigo_treino = pd.read_csv("/content/drive/My Drive/curso_word2vec/treino.csv")
artigo_teste = pd.read_csv("/content/drive/My Drive/curso_word2vec/teste.csv")

In [37]:
nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"])

def tokenizador(texto):
    
    doc = nlp(texto)
    tokens_validos = []
    for token in doc:
        e_valido = not token.is_stop and token.is_alpha
        if e_valido:
            tokens_validos.append(token.text.lower())

    
    return  tokens_validos

texto = "Rio de Janeiro 1231231 ***** @#$ é uma cidade maravilhosa!"
tokens = tokenizador(texto)
print(tokens)


['rio', 'janeiro', 'cidade', 'maravilhosa']


In [38]:
def combinacao_de_vetores_por_soma(palavras, modelo):

    vetor_resultante = np.zeros((1,300))

    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)

        except KeyError:
            pass
                

    return vetor_resultante

vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(1, 300)
[[-8.66313378e-04  3.59719642e-03  2.93452802e-04 -2.86260055e-03
   3.62503121e-03  3.76887160e-03  4.55745749e-03  2.50877853e-03
  -3.19792889e-07 -6.27870869e-03 -3.56720778e-03  1.79750635e-03
  -4.15625982e-06 -1.70109875e-03 -1.72676804e-03 -9.74329305e-04
   9.34839656e-04  1.39443472e-03  8.64906411e-04 -5.56693063e-04
   3.63325002e-03 -1.67594949e-03 -2.57178067e-03  8.93699820e-04
   1.53063333e-03 -1.27390621e-03 -4.39744757e-03  4.06522321e-03
  -1.28605682e-03  3.70716646e-03 -3.96201976e-03  1.57101953e-04
   7.95827305e-04  1.18947986e-03  7.44248246e-04 -4.03729471e-04
  -7.98778143e-04 -3.91400745e-03 -2.95046018e-03 -1.46215329e-03
  -4.12767538e-03  4.43571364e-04  4.19312302e-03  3.84604849e-04
  -5.24176658e-03 -6.08090169e-03 -1.42408081e-03  2.44172418e-03
  -4.82491615e-03  3.41561250e-03  2.76853354e-03 -3.03056929e-03
   1.68141371e-03 -2.06260447e-03  1.91833335e-03 -2.50621836e-03
  -7.41957116e-03  7.53842120e-04 -1.08000799e-03 -2.67847057e-03
 

In [39]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz

matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape) 

(90000, 300)
(20513, 300)


In [40]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):

    RL = LogisticRegression(max_iter = 800)
    RL.fit(x_treino, y_treino)
    categorias = RL.predict(x_teste)
    resultados = classification_report(y_teste, categorias)
    print(resultados)
    
    return RL

RL_cbow = classificador(w2v_modelo_cbow,
                        matriz_vetores_treino_cbow,
                        artigo_treino.category,
                        matriz_vetores_teste_cbow,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.65      0.37      0.47      6103
   cotidiano       0.34      0.52      0.41      1698
     esporte       0.58      0.57      0.57      4663
   ilustrada       0.02      0.39      0.04       131
     mercado       0.62      0.49      0.55      5867
       mundo       0.34      0.52      0.41      2051

    accuracy                           0.48     20513
   macro avg       0.43      0.48      0.41     20513
weighted avg       0.57      0.48      0.50     20513



In [41]:
matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)

RL_sg = classificador(w2v_modelo_sg,
                        matriz_vetores_treino_sg,
                        artigo_treino.category,
                        matriz_vetores_teste_sg,
                        artigo_teste.category)
     

              precision    recall  f1-score   support

     colunas       0.65      0.37      0.47      6103
   cotidiano       0.34      0.52      0.41      1698
     esporte       0.58      0.57      0.57      4663
   ilustrada       0.02      0.39      0.04       131
     mercado       0.62      0.49      0.55      5867
       mundo       0.34      0.52      0.41      2051

    accuracy                           0.48     20513
   macro avg       0.43      0.48      0.41     20513
weighted avg       0.57      0.48      0.50     20513



In [42]:
import pickle

with open("/content/drive/My Drive/curso_word2vec/rl_cbow.pkl", "wb") as f:
    pickle.dump(RL_cbow, f)

In [43]:
with open("/content/drive/My Drive/curso_word2vec/rl_sg.pkl", "wb") as f:
    pickle.dump(RL_sg, f)
